In [2]:
# First cell: Import and Input
import math

# Input parameters
R = "Relation1"  # Name of first relation
S = "Relation2"  # Name of second relation

# Relations params
nR = 10000  # Number of tuples in R
nS = 5000  # Number of tuples in S
bR = 400  # Number of pages of R
bS = 1000  # Number of pages of S

# System params
M = 50  # Number of buffer pages available
time_seek = 0.1  # Time for seeking a block from disk (in seconds)
time_retrieve = 0.01  # Time for retrieving a block from disk (in seconds)
hBTree = 4

print(f"Input parameters set for relations {R} and {S}")


Input parameters set for relations Relation1 and Relation2


# Hash Join 

s : build input 

r : probe input


In [3]:
def hash_join(bR, bS, time_seek, time_retrieve, M, fudge_factor=1.2):
   
    nh = math.ceil((bS / M) * fudge_factor) # ⌈bs / M⌉ * f to prevent skew values

    block_transfers = 3 * (bR + bS) + 4 * nh
    seeks = 2 * (math.ceil(bR / nh) + math.ceil(bS / nh)) + 2 * nh

    #time
    time_cost = (block_transfers * time_retrieve) + (seeks * time_seek)

    print(f"Number of partitions (nh): {nh}")
    print(f"Block transfers: {block_transfers}")
    print(f"Seeks: {seeks}")
    print(f"Total time for hash join: {time_cost:.2f} seconds")
    return time_cost

time_hash_join = hash_join(bR, bS, time_seek, time_retrieve, M)
print(f"Time for Hash Join: {time_hash_join:.2f} seconds")


Number of partitions (nh): 24
Block transfers: 4296
Seeks: 166
Total time for hash join: 59.56 seconds
Time for Hash Join: 59.56 seconds


# Recursive Hash-Join  

In [9]:
def recursive_hash_join(bR, bS, time_seek, time_retrieve, M, fudge_factor=1.2, max_recursion_depth=10, recursion_depth=0):
   
    # Prevent infinite recursion
    if recursion_depth > max_recursion_depth:
        print("Max recursion depth reached, aborting.")
        return 0

    # Calculate number of partitions for S based on the buffer size and fudge factor
    nhS = math.ceil(bS / M * fudge_factor)  # Only nhS matters

    print(f"Partitioning S into {nhS} partitions...")

    # Base case: if S fits into memory, perform the hash join
    if nhS == 1:
        return hash_join(bR, bS, time_seek, time_retrieve, M, fudge_factor)

    # Recursive case: partition further and join
    total_time = 0
    
    for j in range(nhS):
        # Recursively partition and join the sub-relations
        total_time += recursive_hash_join(bR, bS // nhS, time_seek, time_retrieve, M, fudge_factor, max_recursion_depth, recursion_depth + 1)

    return total_time

# Example of calling the recursive hash join
time_recursive_hash_join = recursive_hash_join(bR, bS, time_seek, time_retrieve, M)
print(f"Total time for recursive hash join: {time_recursive_hash_join:.2f} seconds")


Partitioning S into 24 partitions...
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.67 seconds
Partitioning S into 1 partitions...
Number of partitions (nh): 1
Block transfers: 1327
Seeks: 884
Total time for hash join: 101.6